In [1]:
pip install nltk pycld2 wordsegment autocorrect

     |████████████████████████████████| 41.4MB 113kB/s 
     |████████████████████████████████| 4.8MB 35.8MB/s 
     |████████████████████████████████| 1.8MB 34.8MB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp36-cp36m-linux_x86_64.whl size=9833500 sha256=d2b9eb1a5e23dd98b140ea75fb066a9486206816c61b961cb55032aff080a89a
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3dbcfc70498100de11079
  Created wheel for autocorrect: filename=autocorrect-2.1.0-cp36-none-any.whl size=1811979 sha256=3e7bcdc982ebedde84cd5f24be6622697d27337bb2ee50388c564318a253bc02
  Stored in directory: /root/.cache/pip/wheels/d1/33/69/a42db65d595bb685429c788ab45e11aa2a4b6549298ee38e02
Successfully built pycld2 autocorrect


In [2]:
import nltk
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples
twitter_samples.fileids()
positive_tweets = [t for t in twitter_samples.strings("positive_tweets.json")]
negative_tweets = [t for t in twitter_samples.strings("negative_tweets.json")] 

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


In [3]:
print(len(positive_tweets))
print(len(negative_tweets))

5000
5000


In [4]:
train_pos_tweets = positive_tweets[:3500]
test_pos_tweets = positive_tweets[3500:]
train_neg_tweets = negative_tweets[:3500]
test_neg_tweets = negative_tweets[3500:]

In [5]:
print(train_pos_tweets[0])

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)


In [6]:
print(train_neg_tweets[7])

@f0ggstar @stuartthull work neighbour on motors. Asked why and he said hates the updates on search :( http://t.co/XvmTUikWln


In [7]:
print(len(train_pos_tweets))
print(len(test_pos_tweets))
print(len(train_neg_tweets))
print(len(test_neg_tweets))

3500
1500
3500
1500


In [8]:
print(len(train_pos_tweets + train_neg_tweets))
print(len(test_pos_tweets + test_neg_tweets))

7000
3000


In [9]:
# import pycld2 as cld2

# def extract_engTweets(tweets):
#     en_tweets = []
#     for e in tweets:
#       details = cld2.detect(e)
#       # Extracts only the string with lang name i.e: 'en'
#       if details[2][0][1] == 'en': 
#         en_tweets.append(e)
#     return en_tweets

# en_train_pos_tweets = extract_engTweets(train_pos_tweets)
# en_train_neg_tweets = extract_engTweets(train_neg_tweets)
# en_test_pos_tweets = extract_engTweets(test_pos_tweets)
# en_test_neg_tweets = extract_engTweets(test_neg_tweets)
# print(len(train_pos_tweets))
# print(len(en_train_pos_tweets))
# print(len(train_neg_tweets))
# print(len(en_train_neg_tweets))
# print(len(test_pos_tweets))
# print(len(en_test_pos_tweets))
# print(len(test_neg_tweets))
# print(len(en_test_neg_tweets))


In [ ]:
#pre processing
import re
import wordsegment
import autocorrect
from wordsegment import load, segment
load()
from autocorrect import Speller
spell=Speller()
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")
tknzr = TweetTokenizer()

punctuations = '''!()-![]{};:+'"\,<>./?@#$%^&*_~'''
testTweet1='@PERKSOFNIALLJH RTed! Goood Luck :)'
testTweet2='My legs hurt so bad :))))))))))))))))))'
testTweet3='#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris trouble for being top engaged members in my community this week :)'
testTweet4='@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!'

def pre_process (tweets):
  result_tweets = []
  for tweet in tweets:  
    # print(f'raw tweet: {tweet}')
    tweet = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', "", tweet, flags=re.MULTILINE) # to remove links that start with HTTP/HTTPS in the tweet
    tweet = re.sub(r'[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', "", tweet, flags=re.MULTILINE) # to remove other url links
    tweet = re.sub(r"RT", ' ', tweet, flags=re.MULTILINE) #remove 'RT'
    tweet = re.sub(r"#(\w+)", ' ', tweet, flags=re.MULTILINE) #remove '#'
    tweet = re.sub(r"@(\w+)", ' ', tweet, flags=re.MULTILINE) #remove '@'
    tweet = ''.join([i for i in tweet if not i in punctuations]) #remove ponctuations
    tweet = ' '.join(segment(tweet)) #Remove repeated letters
    tweet = ' '.join([spell(w) for w in tweet.split()]) #Avoiding misspellings and slang words
    tweet = re.sub(r"\d", "", tweet) #remove digits
    tweet = tweet.lower()
    tokens = tknzr.tokenize(tweet) #tokenize
    tokens = [word for word in tokens if not word in stopwords.words()]
    tweet = (" ").join(stemmer.stem(token) for token in tokens)
    # print(f'preprocessed tweet: {tweet}')
    result_tweets.append(tweet)
  
  return result_tweets

# proc_en_train_pos_tweets = pre_process([testTweet1,testTweet2,testTweet3,testTweet4])
# proc_en_train_pos_tweets = pre_process([en_train_pos_tweets[0]])
# proc_en_train_neg_tweets = pre_process([en_train_neg_tweets[0]])
proc_train_pos_tweets = pre_process(train_pos_tweets)
proc_train_neg_tweets = pre_process(train_neg_tweets)
proc_test_pos_tweets = pre_process(test_pos_tweets)
proc_test_neg_tweets = pre_process(test_neg_tweets)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
print(len(train_pos_tweets))
print(len(test_pos_tweets))
print(len(train_neg_tweets))
print(len(test_neg_tweets))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=1, max_df=0.75, ngram_range=(1,3), max_features=10000)
features_train_pos = vectorizer.fit_transform(proc_train_pos_tweets).toarray()
features_train_neg = vectorizer.fit_transform(proc_train_neg_tweets).toarray()
features_test_pos = vectorizer.fit_transform(proc_test_pos_tweets).toarray()
features_test_neg = vectorizer.fit_transform(proc_test_neg_tweets).toarray()
# print(f'features_train_pos: {features_train_pos}')
# print(f'type(features_train_pos): {type(features_train_pos)}')

In [ ]:
print(f'features_train_pos.shape: {features_train_pos.shape}')
print(f'features_train_neg.shape: {features_train_neg.shape}')
print(f'features_test_pos.shape: {features_test_pos.shape}')
print(f'features_test_neg.shape: {features_test_neg.shape}')

In [ ]:
import numpy as np
all_tweets = np.concatenate((features_train_pos,features_train_neg,features_test_pos,features_test_neg))
train_features = np.concatenate((features_train_pos,features_train_neg))
test_features = np.concatenate((features_test_pos,features_test_neg))

In [ ]:
## Scoring and labeling the tweets
# First calculate the overall polarity score
# Assume only positive and negative
# Classify via the number of neg/pos words and emoticons

# 'tweet is expected to be an integer iterable' 
def polarity_score(tweet):
  score = 0
  for feature in tweet:
    score += feature
  return score

# 'pol_score' is expected to be a scalar
def determine_sentiment(pol_score):
  # Highly positive: +2 
  if pol_score >= 2 and pol_score <= 4:
    return 2 
  # Moderate positive: +1 
  if pol_score > 0 and pol_score < 2:
    return 1
  # Moderate negative: -1 
  if pol_score > -2 and pol_score < 0:
    return -1 
  # Highly negative: -2 
  if pol_score >= -4 and pol_score <= -2:
    return -2
  if pol_score == 0: 
    return 0
  else:#Tirar depois
    return 2

In [ ]:
## Testing the balancing and scoring
# Classify the tweets
def score_tweets(tweets):
  tweet, length = tweets.shape
  tweets_class = []
  for i in range(tweet):
    score = polarity_score(tweets[i,:])
    sentiment = determine_sentiment(score)
    tweets_class.append(sentiment)
  return tweets_class

all_tweets_classes = np.array(score_tweets(all_tweets))
train_classes = np.array(score_tweets(train_features))
test_classes = np.array(score_tweets(test_features))

In [ ]:
#Table 1 - Polarity of tweets
high_positive = 0
moderate_positive = 0
neutral = 0
moderate_negative = 0
high_negative = 0

for t in all_tweets_classes:
  if t == 2:
    high_positive += 1
  if t == 1:
    moderate_positive += 1
  if t == 0:
    neutral += 1
  if t == -1:
    moderate_negative += 1
  if t == -2:
    high_negative += 1

hp_ratio = (high_positive/all_tweets_classes.size)*100
mp_ratio = (moderate_positive/all_tweets_classes.size)*100
neutral_ratio = (neutral/all_tweets_classes.size)*100
mn_ratio = (moderate_negative/all_tweets_classes.size)*100
hn_ratio = (high_negative/all_tweets_classes.size)*100

print(f'high_positive: {high_positive} ({hp_ratio}%)')
print(f'moderate_positive: {moderate_positive} ({mp_ratio}%)')
print(f'neutral: {neutral} ({neutral_ratio}%)')
print(f'moderate_negative: {moderate_negative} ({mn_ratio}%)')
print(f'high_negative: {high_negative} ({hn_ratio}%)')

In [ ]:
all_tweets_classes[0] == 2 

In [ ]:
# # Decision tree implementation
# from sklearn import tree
# from sklearn.model_selection import GridSearchCV
# from sklearn.datasets import load_iris # remove after real values come from Jamelly
# from sklearn.model_selection import train_test_split # remove after real values come from Jamelly
# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_val_score

# # prepare the cross-validation procedure
# cv = KFold(n_splits=10, random_state=1, shuffle=True)
# ## create model
# #model = LogisticRegression()
# ## evaluate model
# #scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# ## report performance
# #print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

# iris = load_iris()
# X = iris.data
# y = iris.target
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# x = X_train
# y = y_train

# clf = tree.DecisionTreeClassifier()

# clf.fit(x,y)
# clf.predict(X_test)
# print(x.shape)
# print(y.shape)

In [ ]:
train_classes[0:50]

In [ ]:
np.isnan(train_features).any()

In [ ]:
# Decision tree implementation
from sklearn import tree
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(all_tweets, all_tweets_classes, test_size=0.3, random_state=42)

clf = tree.DecisionTreeClassifier()

clf.fit(train_features, train_classes)
y_predict = clf.predict(test_features)

In [ ]:
# 10-fold cross-validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)

scores = cross_val_score(clf, train_features, train_classes, scoring='accuracy', cv=cv, n_jobs=-1)
## report performance
# print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
print(f'scores.shape{scores.shape}')
print(f'scores{scores}')

In [ ]:
print(test_classes.shape)
print(y_predict.shape)
print(test_features.shape)

In [ ]:
from sklearn.metrics import classification_report
# y_true = [0, 1, 2, 2, 2]
# y_pred = [0, 0, 2, 2, 1]
# target_names = ['class 0', 'class 1', 'class 2']
y_true = test_classes
y_pred = y_predict 
target_names = ['Neutral', 'Moderate Positive', 'High Positive']
# target_names = [0, 1, 2]
# print(classification_report(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
# Brief explanation about machine learning validation metrics
# https://medium.com/analytics-vidhya/complete-guide-to-machine-learning-evaluation-metrics-615c2864d916


In [ ]:
## create model
#model = LogisticRegression()
## evaluate model
#scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
## report performance
#print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
# print(f'proc_en_train_pos_tweets: {len(proc_en_train_pos_tweets)}')
# print(f'en_train_pos_classes: {en_train_pos_classes.shape}')

In [ ]:
# Calculate model precision, recall, f1-score and support
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html

In [ ]:
# https://medium.com/swlh/sentiment-classification-for-restaurant-reviews-using-tf-idf-42f707bfe44d

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html